In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
from sklearn.datasets import load_linnerud
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.model_selection import train_test_split
from sklearn.pipeline import FeatureUnion
from physlearn import Regressor
from sklearn.svm import LinearSVC
import os
import cv2
import re

In [2]:
df=pd.read_excel("data_new.xlsx")
df = df.sample(frac=1)
df.head()

,Ароматичность,Алифатичность,Разветвленность,Окисленность,содержание серы,плотность,вязкость,процент неразделяемой эмульсии,класс химии,процент химии,Ароматичность(химия),Алифатичность(химия),Разветвленность(химия),Окисленность(химия),содержание серы(химия),плотность(химия),вязкость(химия),процент неразделяемой эмульсии(химия)
4427,0.85,5.4,0.41,1.9,0.5,921.0,240.0,6.0,4.0,15.0,15.95,13.3,12.51,13.0,5.6,925.1,246.1,11.1
5830,0.05,8.8,0.38,1.6,0.7,875.0,152.0,31.0,5.0,20.0,17.25,17.3,13.58,13.8,5.9,879.6,158.8,36.2
5982,0.79,9.5,0.50,2.6,4.6,614.0,244.0,43.0,5.0,20.0,17.99,18.0,13.70,14.8,9.8,618.6,250.8,48.2
6215,0.72,3.2,0.44,2.2,6.3,1054.0,298.0,17.0,6.0,5.0,6.02,7.8,7.74,8.5,11.6,1056.3,301.3,22.3
2944,0.84,3.8,0.45,2.7,2.3,976.0,270.0,39.0,3.0,10.0,11.64,10.0,10.25,11.5,7.1,979.0,274.6,43.8


In [3]:
# in dataset there are 2 empty rows as delimeter for classification

after_chem_columns = ["Ароматичность(химия)","Алифатичность(химия)","Разветвленность(химия)","Окисленность(химия)"," содержание серы(химия)",
              "плотность(химия)","вязкость(химия)","процент неразделяемой эмульсии(химия)"]
df = df.dropna(axis='rows')

In [4]:
# for 6 classes and each percent rate 5,10,15,20 (percents per)
average_values_petrol = {}

for chem_class in range(1,7):
    inner_dict = {}
    for percent in [5,10,15,20]:
        inner_dict[percent] = df.loc[(df["класс химии"] == chem_class) & 
                                            (df["процент химии"]==percent),after_chem_columns].mean(axis=0)
    average_values_petrol[chem_class] = inner_dict
    
# for 6 classes and each for 5,10,15,20 (percents per) - output to excel file sheets
with pd.ExcelWriter("output.xlsx") as writer:
    for index in average_values_petrol.keys():
        df1 = pd.DataFrame(average_values_petrol[index])
        df1.to_excel(writer,sheet_name=str(index),index=False)
        
# Perfect petrol match after chemistry 
# 7,8043, 11,099, 9,2492, 12,641, 8,2895, 835,22, 234,49, 29,635

Предсказываем нефть после обработки - 8 столбцов значений

In [5]:
# Кусок кода взят отсюда: https://scikit-physlearn.readthedocs.io/en/latest/quick_start.html

bunch = load_linnerud(as_frame=True)  # returns a Bunch instance
X, y = df.iloc[:,:10], df.iloc[:,10:]
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    random_state=42)

transformer_list = [('pca', PCA(n_components=1)),
                    ('svd', TruncatedSVD(n_components=2))]
union = FeatureUnion(transformer_list=transformer_list, n_jobs=-1)

# Select a regressor, e.g., LGBMRegressor from LightGBM,
# with a case-insensitive string.
reg = Regressor(regressor_choice='lgbmregressor',
                pipeline_transform=('tr', union),
                scoring='neg_mean_absolute_error')

# Automatically build the pipeline with final estimator MultiOutputRegressor
# from Sklearn, then exhaustively search over the (hyper)parameters.
search_params = dict(reg__boosting_type=['gbdt', 'goss'],
                     reg__n_estimators=[6, 8, 10, 20])
reg.search(X_train, y_train, search_params=search_params,
           search_method='gridsearchcv')

# Generate predictions with the refit regressor, then
# compute the average mean absolute error.
y_pred = reg.fit(X_train, y_train).predict(X_test)
score = reg.score(y_test, y_pred)
print(round(score['mae'].mean(),2))
# print(score['mae'].mean().round(decimals=2))

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[Pipeline] ................ (step 1 of 2) Processing tr, total=   0.0s
[Pipeline] ............... (step 2 of 2) Processing reg, total=   0.0s
[Pipeline] ................ (step 1 of 2) Processing tr, total=   0.0s
[Pipeline] ............... (step 2 of 2) Processing reg, total=   0.0s
6.19


Предсказываем химию полученной нефти на сгенерированных предсказаниях нефти после химии

In [6]:
chemisrty_classes = X["класс химии"].unique()
percent_rate = X["процент химии"].unique()

data = X.sample(frac=1)[:1]

# Knn after this
prediction_dict = {}
for chm_class in np.sort(chemisrty_classes):
    data["класс химии"] = chm_class
    for rate in np.sort(percent_rate):
        data["процент химии"] = rate
        predictions = reg.predict(data).values
        prediction_dict[f"{chm_class}_{rate}"] = predictions[0]
        
df_predictions = pd.DataFrame(prediction_dict.values())
df_predictions["class"] = [key.split('_')[0] for key in prediction_dict.keys()]
df_predictions["%rate"] = [key.split('_')[1] for key in prediction_dict.keys()]

In [7]:
# Забиваем идеальную нефть и вытаскиваем ближайшего соседа - то есть класс нефти и процент
# Это обратный подход, то есть мы как будто бы находим на самом деле какой класс и процент использовать,
# чтобы максимально приблизиться к "Идеальному соотншению парметров в нефти"

# targets - это слияние класса и процента химии

# 7,8043, 11,099, 9,2492, 12,641, 8,2895, 835,22, 234,49, 29,635
X = df_predictions.drop(columns=["class","%rate"])
y = df_predictions[["class","%rate"]]
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X, y)
perfect_petrol = [7.8043, 11.099, 9.2492, 12.641, 8.2895, 835.22, 234.49, 29.635]
print(neigh.predict([perfect_petrol]))

[['1.0' '10.0']]
